   # Implementação básica de Rede Neural Artificial

In [1]:
import numpy as np

In [2]:
class RNA:
    
    
    def __init__(self, dims_camada_entrada:list, learning_rate=0.0075, num_iterations=100):
        self.parameters = {} 
        self.custos = [] 
        self.dims_camada_entrada = dims_camada_entrada
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        
        
    def inicializa_parametros(self, dims_camada_entrada:list) -> dict:
        """
        Função para Inicialização randômica dos parametros do modelo.
        Inicializa parametros aleatórios para os pesos.
        
        Arg:
            dims_camada_entrada (list): [total de variáveis, nº neuronio camada 1, 
                                         nº neuronio camada 2,
                                         ..., 
                                         nº neuronio camada n,
                                         ]
        Retorna: 
            dict: dicionário com os valores iniciais dos pesos para cada camada do neurônio.
        """
        comp = len(dims_camada_entrada)
        for i in range(1, comp):
            self.parameters["W" + str(i)] = np.random.randn(dims_camada_entrada[i], dims_camada_entrada[i - 1]) * 0.01
            self.parameters["b" + str(i)] = np.zeros((dims_camada_entrada[i], 1))
        return self.parameters
    
    
    def sigmoid(self, Z:float) -> tuple:
        """
        Função sigmóide: utilizada para ativação da ultima camada
        Converte qualquer valor real de Z para valores no intervalo de 0 e 1.
        
        Args:
            Z (float): qualquer número pertencente ao conjunto dos nº reais.
        
        Retorna:
            tupla : contendo o valor transformado pela função e o próprio valor de entrada
        """
        A = 1 / (1 + np.exp(-Z))
        return A, Z
    
    
    def relu(self, Z:float) -> tuple:
        """
        Função de ativação ReLu (Rectified Linear Unit): Ativação camadas intermediárias
        Converte qualquer valor real de Z para 0 quando Z menor que 0,
        e retorna a imagem de Z para valores maiores que 0. 
        
        Args:
            Z (float): qualquer número pertencete ao conjunto dos nº reais
        
        Retorna:
            tupla: Valor da função relu, valor de entrada Z.
        """
        A = abs(Z * (Z > 0))
        return A, Z
 

    def linear_activation(self, A:np.ndarray, W:np.ndarray, b:np.ndarray) -> tuple:
        """
        Ativação linear
        Realiza a operação de ativação por camada
        
        Args: 
            A (numpy.ndarray): matriz com os dados de entrada da camada
            W (numpy.ndarray): matriz de pesos da camada
            b (numpy.ndarray): bias
        
        Retorna:
            tupla: contendo o valor de ativação e das memórias das matrizes dos pesos. 
        """
        Z = np.dot(W, A) + b
        cache = (A, W, b)
        return Z, cache
    
    
    def forward(self, A_prev: np.ndarray, W:np.ndarray, b:np.ndarray, activation:str) -> tuple:
        """
        Aplica a função de ativação nos resultados da multiplcaçãdo dos dados pelos coeficientes
        
        Args: 
            A_prev (numpy.ndarray): Dados entrada prévios;
            W (numpy.ndarray): Matriz de coeficientes da camada;
            b (numpy.ndarray): bias da camada;
            activation (str): tipo da função de ativação "sigmoid" ou "relu"
        
        Retorna:
            tupla: A: os resultados dos valores passado pela função de ativação;
                   cache: mémoria dos dados iniciais, matriz de coeficiente e bias
        """
        if activation == "sigmoid":
            Z, linear_cache = self.linear_activation(A_prev, W, b)
            A, activation_cache = self.sigmoid(Z)   
        elif activation == "relu":
            Z, linear_cache = self.linear_activation(A_prev, W, b)
            A, activation_cache = self.relu(Z)
        cache = (linear_cache, activation_cache)
        return A, cache

    
    def forward_propagation(self, X:np.ndarray, parameters:dict) -> tuple:
        """
        Realiza as multiplicações de matrizes de cada camada da rede.
        
        Args:
            X (numpy.ndarray): Valores dos dados de treinamento;
            parameters (dict): dicionario das matrizes de coeficientes e bias
        
        Retorna:
            tupla: A_last -> valores de saida da última camada;
                            caches -> lista com os valores calculados nas camadas internas;
        """
        caches = []
        A = X
        L = len(parameters) // 2
        for i in range(1, L):
            A_prev = A
            A, cache = self.forward(A_prev, parameters["W" + str(i)], parameters["b" + str(i)], activation = "relu")            
            caches.append(cache)        
        A_last, cache = self.forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation = "sigmoid")        
        caches.append(cache)
        return(A_last, caches)
    
    
    def calcula_custo(self, A_last:np.ndarray, Y:np.ndarray) -> np.ndarray:
        """
        Calcula a função custo dado a comparação de valores previstos e valores de treinamento reais
        
        Args: 
            A_last (numpy.ndarray): Matriz da última camada contendo as estimativas do modelo.
            Y (numpy.ndarray): valores reais dos dados de treinamento
        
        Retorna: 
            numpy.ndarray: valor da função custo
        """
        # Ajusta o shape de Y para obter seu comprimento (total de elementos)
        m = Y.shape[1]
        # Calcula o custo comparando valor real e previso (função logit para custo)
        custo = (-1 / m) * np.sum((Y * np.log(A_last)) + ((1 - Y) * np.log(1 - A_last)))
        # Ajusta o shape do custo
        custo = np.squeeze(custo)
        return(custo)
    

    def sigmoid_backward(self, da:float, Z:float) -> float:
        """
        Função sigmoid para o backpropagation ( Derivada da função sigmoid)
        Calcula a derivada da função de ativação sigmoid, sua taxa de variação.
        
        Args:
            da (float): derivada da previsão final da rede (feita ao final do Forward Propagation).
            Z (float): valores da multiplicação dos pesos pelos dados da camada.
        
        Retorna: 
            float: taxa de variação da função sigmoid.
        """
        dg = (1 / (1 + np.exp(-Z))) * (1 - (1 / (1 + np.exp(-Z))))
        dz = da * dg # regra da cadeia*
        return dz
    
    
    def relu_backward(self, da:float, Z:float) -> float:
        """
        Função relu para o backpropagation
        Calcula a derivada da função de ativação relu
        
        Args: 
            da (float): derivada da previsão final da rede (feita ao final do Forward Propagation).
            Z (float): valores da multiplicação dos pesos pelos dados da camada.
            
        Retorna: 
            float: taxa de variação da função relu.
        """
        dg = 1 * ( Z >= 0)
        dz = da * dg # regra da cadeia*
        return dz
    
    
    def linear_backward_function(self, dz:float, cache:tuple) -> tuple:
        """
        Ativação linear para o backpropagation
        Calcula as derivadas da operação backward propragation
        
        Args: 
            dz (float): taxa de variação da função de ativação
            cache (tupla): (linear_cache, activation_cache) 
                            linear_cache -> resultado da multiplicação dos pesos pelos dados da camada.
                            activation_cache -> resultado da função de ativação dado o linear_cache como entrada.
        
        Retorna: 
            tupla: derivada da operação (W*dz), derivada da matriz peso W (dz*A_prev), derivada do bias.
        
        """
        A_prev, W, b = cache
        m = A_prev.shape[1]
        dW = (1 / m) * np.dot(dz, A_prev.T)
        db = (1 / m) * np.sum(dz, axis = 1, keepdims = True)
        dA_prev = np.dot(W.T, dz)
        return dA_prev, dW, db


    def linear_activation_backward(self, dA:float, cache:tuple, activation: str) -> tuple:
        """
        Define o tipo de ativação ( relu ou sigmoid).
        
        Args:
            dA (float): derivada da previsão final da rede (feita ao final do Forward Propagation).
            cache (tuple): cache mémoria dos dados iniciais, matriz de coeficiente e bias.
            activation (str): Define a função de ativação "relu" ou "sigmoid".
        
        Retorna:
            tupla: derivada da operação (W*dz), derivada da matriz peso W (dz*A_prev), derivada do bias.
        """
        linear_cache, activation_cache = cache
        if activation == "relu":
            dZ = self.relu_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward_function(dZ, linear_cache)
        if activation == "sigmoid":
            dZ = self.sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward_function(dZ, linear_cache)
        return dA_prev, dW, db


    def backward_propagation(self, AL:np.ndarray, Y:np.ndarray, caches: list) -> dict:
        """
        Calcula os gradientes para atualização dos pesos
        
        Args:
            AL (numpy.ndarray): matriz das previsões realizadas pelo foward.
            Y (numpy.ndarray): matriz dos resultados reais.
            caches (list): lista com os valores calculados nas camadas internas;
        
        Retorna:
            dict: Dicionário contendo os gradientes dos dados, dos pesos e do bias.
        """
        grads = {}
        L = len(caches)
        m = AL.shape[1]
        Y = Y.reshape(AL.shape)
        dAL = -((Y / AL) - ((1 - Y) / (1 - AL)))
        current_cache = caches[L - 1]
        grads["dA" + str(L - 1)], grads["dW" + str(L)], grads["db" + str(L)] = self.linear_activation_backward(dAL, current_cache, activation = "sigmoid")
        for l in reversed(range(L - 1)):
            current_cache = caches[l]
            dA_prev, dW, db = self.linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
            grads["dA" + str(l)] = dA_prev
            grads["dW" + str(l + 1)] = dW
            grads["db" + str(l + 1)] = db        
        return grads

    
    def atualiza_pesos(self, parameters:dict, grads:dict, learning_rate:float) -> dict:
        """
        Função de atualização de pesos
        Realiza a atualização dos pesos em cada camada da rede.
        
        Args:
            parameters (dict): dicionário contendo todas matrizes de pesos.
            grads (dict): dicionário contendo todas as matrizes dos gradientes calculados. 
            learning_rate (float): taxa de aprendizado.
        
        Retorna:
            dict: dicionário contendo as matrizes de pesos atualizados pelo gradiente do backpropagation.
        
        """
        L = len(parameters)//2
        for l in range(L):
            parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - (learning_rate * grads["dW" + str(l + 1)])
            parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - (learning_rate * grads["db" + str(l + 1)])
        return parameters
    
    
    def fit(self, X:np.ndarray, Y:np.ndarray) -> str:
        """
        Executa o treinamento do modelo da Rede Neural Artificial.
        
        Args:
            X (numpy.ndarray): Dados treinamento variáveis independentes.
            Y (numpy.ndarray): Dados treinamento variáveis dependentes.
            dims_camada_entrada: Dimensões das camadas e estruturas de Neurônios.
            learning_rate (float): Taxa de apresendizado do modelo.
            num_iterations (int): Quantidade de iterações para o treinamento do modelo.
            
        Retorna: 
            tupla: dicionário dos parametros treinados, lista dos custos por iteração de treinamento. 
        """
        print('\n Iniciando o treinamento. \n')
        parametros = self.inicializa_parametros(self.dims_camada_entrada)
        for i in range(self.num_iterations):
            AL, caches = self.forward_propagation(X, parametros)
            # Calcula o custo
            custo = self.calcula_custo(AL, Y)
            # Etapa Backward Propagation (Atualização dos pesos)
            gradientes = self.backward_propagation(AL, Y, caches)
            # Atualiza os pesos
            parametros = self.atualiza_pesos(parametros, gradientes, self.learning_rate)
            # Print do valor intermediário do custo
            if i % 10 == 0:
                print("Custo Após " + str(i) + " iterações é " + str(custo))
                self.custos.append(custo)
        return 'Treinamento Concluido!'
    
    
    def predict(X:np.array, parametros:dict) -> tuple:
        """
        Funções para fazer as previsões.
        Calcula as previsões.
        
        Args:
            X (numpy.array): Variaveis independentes.
            parametros (dict) : Dicionário contendo as matrizes dos pesos de cada camada.
        
        Returns: 
            tuple : Valor Previsto, dicionário contendo os parametros do modelo. 
        """
        AL, caches = forward_propagation(X, parametros)
        return AL

## Aplicação da Rede Neural para classificação de células malignas/benignas

In [3]:
# Imports
import sklearn
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
# Carregamos o dataset
dados = pd.DataFrame(columns = load_breast_cancer()["feature_names"], data = load_breast_cancer()["data"])

In [5]:
dados.shape

(569, 30)

In [6]:
dados.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
target = load_breast_cancer()['target']
target.shape

(569,)

In [8]:
# Extraindo os labels
# Dicionário para os labels
labels = {}

# Nomes das classes da variável target
target_names = load_breast_cancer()["target_names"]

# Mapeamento
for i in range(len(target_names)):
    labels.update({i:target_names[i]})

In [9]:
labels

{0: 'malignant', 1: 'benign'}

In [10]:
# Preparando os dados das variáveis preditoras, convertendo de dataframe pandas para array numpy
X = np.array(dados)

In [11]:
X.shape

(569, 30)

In [12]:
# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.15, shuffle=True)

In [13]:
X_train.shape

(483, 30)

In [14]:
y_test.shape

(86,)

In [15]:
# Ajustando os dados de entrada e tbm os dados alvo "target" 
X_train, X_test = X_train.T, X_test.T
y_train, y_test = y_train.reshape(1, len(y_train)), y_test.reshape(1,len(y_test))
print(f'shape dados de treino X:{X_train.shape}')
print(f'shape dados de treino Y:{X_test.shape}')


shape dados de treino X:(30, 483)
shape dados de treino Y:(30, 86)


In [16]:
# Lista com as dimensções de entrada e número de neuronios
# Contém 4 camadas, sendo o primeiro com 50 neuronios, 20, 5 e 1 neuronio de saida.
dims_camada_entrada = [X_train.shape[0],50 ,20, 10, 1]

In [17]:
# Treinamento do modelo
modelo = RNA(dims_camada_entrada, learning_rate=0.0075, num_iterations=1000)
modelo.fit(X_train, y_train)


 Iniciando o treinamento. 

Custo Após 0 iterações é 0.6931547615057043
Custo Após 10 iterações é 0.6917558224163596
Custo Após 20 iterações é 0.6904082475302551
Custo Após 30 iterações é 0.6891101401170953
Custo Após 40 iterações é 0.6878591319924102
Custo Após 50 iterações é 0.6866536764141602
Custo Após 60 iterações é 0.6854927885523366
Custo Após 70 iterações é 0.684374898675531
Custo Após 80 iterações é 0.6832985832550537
Custo Após 90 iterações é 0.6822620664188418
Custo Após 100 iterações é 0.6812636433210099
Custo Após 110 iterações é 0.6803018698149724
Custo Após 120 iterações é 0.679375256346225
Custo Após 130 iterações é 0.6784824975509055
Custo Após 140 iterações é 0.6776223485056415
Custo Após 150 iterações é 0.6767935529166409
Custo Após 160 iterações é 0.6759949147136511
Custo Após 170 iterações é 0.6752252849309219
Custo Após 180 iterações é 0.6744835729106484
Custo Após 190 iterações é 0.6737687018612252
Custo Após 200 iterações é 0.6730796566706646
Custo Após 210 ite

'Treinamento Concluido!'